In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

## Load Stuff

In [4]:
raw_data = load_json('data_v2.json')

In [5]:
raw_data.keys()

dict_keys(['source', 'belts', 'pipes', 'buildings', 'miners', 'items', 'fluids', 'recipes', 'resources'])

In [6]:
key_name_fn = lambda x: x.lower().replace(' ', '-')

In [7]:
building_df = pd.DataFrame(raw_data['buildings'])

In [8]:
item_df = pd.DataFrame.from_dict(raw_data['items'])

In [9]:
fluid_df = pd.DataFrame.from_dict(raw_data['fluids'])

In [10]:
resource_df = pd.DataFrame.from_dict(raw_data['resources'])

In [11]:
recipe_df = pd.DataFrame.from_dict(raw_data['recipes'])

## Process recipe data

### Add power info

In [12]:
recipe_category_to_power = building_df.set_index('category')['power'].to_dict()
# average of fluctuating
recipe_category_to_power.update({'converting' : 250, 'encoding' : 1000})
recipe_df['power_rate'] = recipe_df['category'].map(recipe_category_to_power)

In [13]:
overclock = 'base'
sloop = 'base'

if not overclock in {'max', 'base'} and 'sloop' in {'max', 'base'}:
    raise ValueError

In [14]:
# 2.5**1.321928 from: https://satisfactory.wiki.gg/wiki/Clock_speed

if overclock == 'max':
    recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
    recipe_df['products'] = recipe_df['products'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
    recipe_df['power_rate'] = recipe_df['power_rate'] * 2.5**1.321928
if sloop == 'max':
    recipe_df['products'] = recipe_df['products'].apply(lambda x: [[y[0], y[1] * 2] for y in x])
    recipe_df['power_rate'] = recipe_df['power_rate'] * 4
    

### Recipes for Base items - e.g. ore

In [15]:
base_items = ['Iron Ore', 'Limestone', 'Copper Ore', 'Bauxite', 'Caterium Ore', 'Coal','Raw Quartz', 'Sulfur', 'Uranium'] + ['Nitrogen Gas', 'Water', 'Crude Oil', 'SAM']
base_key_items = [key_name_fn(x) for x in base_items]
base_key_items_set = set(base_key_items)

In [16]:
list(recipe_df)

['name',
 'key_name',
 'category',
 'time',
 'ingredients',
 'products',
 'power_range',
 'power_rate']

In [17]:
base_recipes = pd.DataFrame([[it, key_it, 'base', 0.0, [], [[key_it, 1]], np.nan, 0] for (it, key_it) in zip(base_items, base_key_items)], columns = list(recipe_df))
base_recipes.shape

(13, 8)

In [18]:
recipe_df = pd.concat([recipe_df, base_recipes])

In [19]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
8,Uranium,uranium,base,0.0,[],"[[uranium, 1]]",NaN,0
9,Nitrogen Gas,nitrogen-gas,base,0.0,[],"[[nitrogen-gas, 1]]",NaN,0
10,Water,water,base,0.0,[],"[[water, 1]]",NaN,0
11,Crude Oil,crude-oil,base,0.0,[],"[[crude-oil, 1]]",NaN,0


### load/process points data

In [20]:
points_df = pd.read_html('https://satisfactory.fandom.com/wiki/AWESOME_Sink')[3]

In [21]:
points_df

,Points,Items
0,1,"FICSIT Coupon (For the first time, the Cyber W..."
1,1,"FICSMAS Tree Branch, FICSMAS Decoration, Blue ..."
2,2,"Iron Ingot, Screw, Limestone"
3,3,"Coal, Copper Ore, Leaves"
4,4,Iron Rod
...,...,...
98,257 312,Pressure Conversion Cube
99,413 920,Hover Pack
100,543 424,Nuclear Pasta
101,543 632,Assembly Director System


In [22]:
# split stuff as list by comma & explode
points_df['Items'] = points_df['Items'].apply(lambda x: x.split(','))
points_df = points_df.explode('Items').reset_index(drop=True).iloc[5:]
points_df.rename({'Points' : 'points', 'Items' : 'item'}, axis=1, inplace=True)
points_df['item'] = points_df['item'].apply(lambda x: x.strip())
points_df['points'] = points_df['points'].apply(lambda x: int(x.replace(' ', '')))

In [23]:



# ['alt-distilled-silica', 'alt-heavy-oil-residue', 'alt-polymer-resin', 'alumina-solution', 'ballistic-warp-drive', 'crude-oil', 'dark-matter-crystal', 'dark-matter-residue', 'diamonds', 'encased-plutonium-cell', 'excited-photonic-matter', 'ficsite-ingot', 'ficsite-trigon', 'ficsonium', 'ficsonium-fuel-rod', 'fuel', 'hatcher-remains', 'hog-remains', 'ionized-fuel', 'liquid-biofuel', 'neural-quantum-processor', 'nitric-acid', 'nitrogen-gas', 'non-fissile-uranium', 'packaged-ionized-fuel', 'packaged-rocket-fuel', 'plutonium-pellet', 'plutonium-waste', 'power-shard', 'reanimated-sam', 'rocket-fuel', 'sam', 'sam-fluctuator', 'singularity-cell', 'spitter-remains', 'stinger-remains', 'sulfuric-acid', 'superposition-oscillator', 'time-crystal', 'turbofuel', 'uranium-waste', 'water']


In [24]:
missing_points = [
    [597652, 'AI Expansion Server'],
    [210, 'Alien Power Matrix'],
    [2895334, 'Ballistic Warp Drive'],
    [1780, 'Dark Matter Crystal'],
    [240, 'Diamonds'],
    [1936, 'Ficsite Ingot'],
    [1291, 'Ficsite Trigon'],
    [248034, 'Neural Quantum Processor'],
    [5246, 'Packaged Ionized Fuel'],
    [1028, 'Packaged Rocket Fuel'],
    [160, 'Reanimated SAM'],
    [20, 'SAM'],
    [1968, 'SAM Fluctuator'],
    [114675, 'Singularity Cell'],
    [37292, 'Superposition Oscillator'],
    [960, 'Time Crystal']
]
points_df = pd.concat([points_df, pd.DataFrame.from_records(missing_points, columns = list(points_df))], axis=0).reset_index(drop=True)

In [25]:
garbage = ["Actual Snow",
"Candy Cane",
"FICSMAS Gift",
"Sweet Fireworks",
"Sparkly Fireworks",
"Copper FICSMAS Ornament",
"FICSMAS Ornament Bundle",
"Iron FICSMAS Ornament",
"Snowball",
"Red FICSMAS Ornament",
"FICSMAS Wonder Star",
"Fancy Fireworks",
"FICSMAS Bow",
'Portable Miner']

In [26]:
points_df = points_df[points_df['item'].apply(lambda x: not x in garbage)].reset_index(drop=True)

In [27]:
points_df['key_name'] = points_df['item'].apply(key_name_fn)

In [28]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
8,Uranium,uranium,base,0.0,[],"[[uranium, 1]]",NaN,0
9,Nitrogen Gas,nitrogen-gas,base,0.0,[],"[[nitrogen-gas, 1]]",NaN,0
10,Water,water,base,0.0,[],"[[water, 1]]",NaN,0
11,Crude Oil,crude-oil,base,0.0,[],"[[crude-oil, 1]]",NaN,0


In [29]:
points_df

,points,item,key_name
0,1,Iron Ore,iron-ore
1,2,Iron Ingot,iron-ingot
2,2,Screw,screw
3,2,Limestone,limestone
4,3,Coal,coal
...,...,...,...
134,20,SAM,sam
135,1968,SAM Fluctuator,sam-fluctuator
136,114675,Singularity Cell,singularity-cell
137,37292,Superposition Oscillator,superposition-oscillator


### Filter out unwanted things form poitns & recipes

In [30]:
ignore_things = [key_name_fn(x) for x in ['Hog Remains', 'Spitter Remains', 'Stinger Remains', 'Hatcher Remains', 'Leaves', 'Wood', 'Mycelia', 'Yellow Power Slug', 'Purple Power Slug', 'Blue Power Slug', 'Automated Miner']]

In [31]:
print(recipe_df.shape)
recipe_df = recipe_df[recipe_df['products'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[recipe_df['ingredients'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[~recipe_df['key_name'].isin(ignore_things)].reset_index(drop=True)
recipe_df.shape

(285, 8)


(272, 8)

In [32]:
print(points_df.shape)
points_df = points_df[~points_df['key_name'].isin(ignore_things)].reset_index(drop=True)
points_df.shape

(139, 3)


(136, 3)

### Make points data & recipe data joinable

In [33]:
points_rename_mapper = {
    'encased-uranium-cell' : 'uranium-cell',
#     'biomass-(leaves)' : 'biomass-from-leaves',
#     'biomass-(mycelia)' : 'biomass-from-mycelia',
#     'biomass-(wood)' : 'biomass-from-wood',
    'gas-filter' : 'filter',
    'electromagnetic-control-rod' : 'em-control-rod',
    
}
points_df['key_name'] = points_df['key_name'].apply(lambda x: points_rename_mapper.get(x, x))

In [34]:
sys.exit()

NameError: name 'sys' is not defined

### Validate that everything is good
We need:  
* all ingredients/products that should be scoreable are scoreable
* WHAT ELSE DO WE NEED???

In [41]:
NO_SINK_ENTRY = set([key_name_fn(x) for x in ['Alien Protein', 'Encased Plutonium Cell', 'Ficsonium', 'Ficsonium Fuel Rod', 'Non Fissile Uranium', 'Plutonium Pellet', 'Plutonium Waste', 'Power Shard', 'Uranium Waste']] + fluid_df['key_name'].tolist())

In [42]:
all_ingredients = set(recipe_df['ingredients'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())
all_products = set(recipe_df['products'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())

In [43]:
all_scoreables = set(points_df['key_name'].tolist())

In [44]:
len(all_ingredients), len(all_products), len(all_scoreables)

(124, 139, 136)

In [45]:
fluid_df

,name,key_name,tier
0,Water,water,3
1,Crude Oil,crude-oil,5
2,Fuel,fuel,5
3,Turbofuel,turbofuel,6
4,Heavy Oil Residue,heavy-oil-residue,5
5,Liquid Biofuel,liquid-biofuel,5
6,Alumina Solution,alumina-solution,7
7,Sulfuric Acid,sulfuric-acid,7
8,Nitrogen Gas,nitrogen-gas,8
9,Nitric Acid,nitric-acid,8


In [47]:
missing_scores = (all_ingredients | all_products) - all_scoreables - NO_SINK_ENTRY

In [48]:
if len(missing_scores):
    print('MISSING: ', missing_scores)
    raise ValueError

### Setup/Solve LP!

In [49]:
"""
how on earth do I make a linear program again???

scipy API:
    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)
    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable

    min c.T @ X
    s.t.
    A_{ub}x <= b_{ub}
    A_{eq}x == b_{eq}
    lb <= x <= ub


X: baseline: how many of each item am I making
C: associated score

Issue:
    items get used as ingredients don't get scored, but you'd want to use them potentially
    So need to duplicate all items:
        [items1, items2]
        items

"""


"\nhow on earth do I make a linear program again???\n\nscipy API:\n    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)\n    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable\n\n    min c.T @ X\n    s.t.\n    A_{ub}x <= b_{ub}\n    A_{eq}x == b_{eq}\n    lb <= x <= ub\n\n\nX: baseline: how many of each item am I making\nC: associated score\n\nIssue:\n    items get used as ingredients don't get scored, but you'd want to use them potentially\n    So need to duplicate all items:\n        [items1, items2]\n        items\n\n"

In [55]:
base_inputs_test = {
    'iron-ore' : 1000,
}
base_inputs_irl = {
    'coal' : 42300,
    'crude-oil' : 12600,
    'nitrogen-gas' : 12000,
    'bauxite' : 12300,
    'copper-ore': 36900,
    'caterium-ore' : 15000,
    'iron-ore' : 92100,
    'uranium' : 2100,
    'raw-quartz' : 13500,
    'sam' : 10200,
    'limestone' : 69300,
    'sulfur' : 10800,
    'water' : 131250,
    'excited-photonic-matter' : 10000,
}

for key in base_items:
    key = key.lower().replace(' ', '-')
    if key in base_inputs_irl:
        continue
    base_inputs_irl[key] = 0


base_inputs_irl = base_inputs_irl | {x_processed : 0 for x in base_items if (not (x_processed:=x.lower().replace(' ', '-'))) in base_inputs_irl}

In [50]:
from scipy.optimize import linprog

In [51]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
267,Uranium,uranium,base,0.0,[],"[[uranium, 1]]",NaN,0
268,Nitrogen Gas,nitrogen-gas,base,0.0,[],"[[nitrogen-gas, 1]]",NaN,0
269,Water,water,base,0.0,[],"[[water, 1]]",NaN,0
270,Crude Oil,crude-oil,base,0.0,[],"[[crude-oil, 1]]",NaN,0


In [53]:
points_df

,points,item,key_name
0,1,Iron Ore,iron-ore
1,2,Iron Ingot,iron-ingot
2,2,Screw,screw
3,2,Limestone,limestone
4,3,Coal,coal
...,...,...,...
131,20,SAM,sam
132,1968,SAM Fluctuator,sam-fluctuator
133,114675,Singularity Cell,singularity-cell
134,37292,Superposition Oscillator,superposition-oscillator


In [109]:
def lp_solve(pdf, rdf, base_inputs):
    #pdf = points_df, rdf=recipe_df
    pdf = pdf.copy()
    rdf = rdf.copy()
    point_mapper = pdf.set_index('key_name')['points'].to_dict()
    
    
    all_recipe_items = sorted(list(set(rdf['ingredients'].explode().dropna().apply(lambda x: x[0]).tolist() + rdf['products'].explode().dropna().apply(lambda x: x[0]).tolist())))
    
    trash_df = pd.DataFrame.from_records([[f'{item} Trash', f'{item}-trash', 'sink', 0, [[item, 1]], [], np.nan, 0] for item in all_recipe_items if item in point_mapper], columns = list(rdf))
    rdf = pd.concat([rdf, trash_df], axis=0).reset_index(drop=True)    
    rdf['id'] = np.arange(rdf.shape[0])

    
#     display(pdf)
#     display(rdf)
#     sys.exit()
    
    
    recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    inv_recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    
    constraint_matrix = np.zeros((len(all_recipe_items), rdf.shape[0]))
    constraint_values = np.zeros(len(all_recipe_items))
    
    def build_recipe_constraints(row):
        ingr_list = row['ingredients']
        products = row['products']
        col_idx = row['id']
        
        
        if not len(ingr_list):
            base_amount = base_inputs[row['key_name']]
            constraint_values[recipe_constraint_row_order[row['key_name']]] = base_amount
        else:
            for ingr, qty in ingr_list:
                constraint_matrix[recipe_constraint_row_order[ingr], col_idx] = qty
            if len(products):
                for prod, qty in products:
                    constraint_matrix[recipe_constraint_row_order[prod], col_idx] = -qty
                
    
    rdf.apply(build_recipe_constraints, axis=1)
            
            
#     print(constraint_matrix)
#     print(constraint_values)
    
    maximize_vec = np.array([0] *(rdf.shape[0] - len(all_recipe_items)) + [point_mapper[item] if item in point_mapper else -10_000 for item in all_recipe_items])
    

    print(constraint_matrix.shape, constraint_values.shape, maximize_vec.shape)
    lp_res = linprog(-maximize_vec, A_ub=None, b_ub=None, A_eq=constraint_matrix, b_eq=constraint_values, bounds=(0, None), method='highs', callback=None, options=None, x0=None, integrality=3)
    
    values = lp_res.x
    
    rdf['qty_produce'] = values
    
    display(rdf.sort_values('qty_produce', ascending=False))
    
    return lp_res
    

In [110]:
eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot', 'iron-plate', 'iron-rod'})].reset_index(drop=True).copy()
eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-plate', 'iron-ore', 'iron-rod'})].reset_index(drop=True).copy()

In [111]:
res = lp_solve(eg_points, eg_recipes, base_inputs_test)

(4, 8) (4,) (8,)


,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4,0,1000.0
2,Iron Rod,iron-rod,crafting1,4.0,"[[iron-ingot, 1]]","[[iron-rod, 1]]",NaN,4,2,1000.0
7,iron-rod Trash,iron-rod-trash,sink,0.0,"[[iron-rod, 1]]",[],NaN,0,7,1000.0
1,Iron Plate,iron-plate,crafting1,6.0,"[[iron-ingot, 3]]","[[iron-plate, 2]]",NaN,4,1,0.0
3,Iron Ore,iron-ore,base,0.0,[],"[[iron-ore, 1]]",NaN,0,3,0.0
4,iron-ingot Trash,iron-ingot-trash,sink,0.0,"[[iron-ingot, 1]]",[],NaN,0,4,0.0
5,iron-ore Trash,iron-ore-trash,sink,0.0,"[[iron-ore, 1]]",[],NaN,0,5,0.0
6,iron-plate Trash,iron-plate-trash,sink,0.0,"[[iron-plate, 1]]",[],NaN,0,6,0.0


In [112]:
res = lp_solve(points_df, recipe_df, base_inputs_irl)

(140, 388) (140,) (388,)


,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4,0,None
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16,1,None
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30,2,None
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30,3,None
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4,4,None
...,...,...,...,...,...,...,...,...,...,...
383,uranium Trash,uranium-trash,sink,0.0,"[[uranium, 1]]",[],NaN,0,383,None
384,uranium-cell Trash,uranium-cell-trash,sink,0.0,"[[uranium-cell, 1]]",[],NaN,0,384,None
385,uranium-fuel-rod Trash,uranium-fuel-rod-trash,sink,0.0,"[[uranium-fuel-rod, 1]]",[],NaN,0,385,None
386,versatile-framework Trash,versatile-framework-trash,sink,0.0,"[[versatile-framework, 1]]",[],NaN,0,386,None


In [113]:
res

       message: The problem is unbounded or infeasible. (HiGHS Status 9: model_status is Primal infeasible or unbounded; primal_status is At lower/fixed bound)
       success: False
        status: 4
           fun: None
             x: None
           nit: -1
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None

In [ ]:
# recipe_df[recipe_df['products'].apply(lambda x: any(['compacted' in y[0] for y in x]))]

In [ ]:
# ingredient_renaming = {
#     'compacted-coal' : 'alt-coal',
#     'heavy-oil-residue' : 'alt-heavy-oil-residue',
#     'alien-protien' : 'biomass-from-alien-protien',
#     'polymer-resin' : 'alt-polymer-resin',
#     'dissolved-silica' : 'alt-distilled-silica'
# }
# recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: [[ingredient_renaming.get(y[0], y[0]), y[1]] if len(y) else y for y in x])
# recipe_df['products'] = recipe_df['products'].apply(lambda x: [[ingredient_renaming.get(y[0], y[0]), y[1]] if len(y) else y for y in x])

In [ ]:
list(recipe_df)

In [ ]:
# recipe_df = recipe_df[~(recipe_df['name'].apply(lambda x:  'Power Shard' in x))].reset_index(drop=True)

In [ ]:
# key_renamer = {
#     'hog-protein' : 'alien-protein',
#     'spitter-protein' : 'alien-protein',
#     'stinger-protein' : 'alien-protein',
#     'hatcher-protein' : 'alien-protein',
#     'biomass-from-wood' : 'biomass',
#     'biomass-from-leaves' : 'biomass',
#     'biomass-from-mycelia' : 'biomass',
#     'biomass-from-alien-protein' : 'biomass',
#     'ficsite-ingot-iron' : 'ficsite-ingot',
#     'ficsite-ingot-caterium' : 'ficsite-ingot',
#     'ficsite-ingot-aluminum' : 'ficsite-ingot'
# }
# recipe_df['key_name'] = recipe_df['key_name'].apply(lambda x: key_renamer.get(x, x))

In [ ]:

all_recipe_key_names = set(recipe_df['key_name'])

In [ ]:
len(all_ingredients), len(all_products)

In [ ]:
points_df['match'] = points_df['key_name'].apply(lambda x: x in all_products)

In [ ]:
# all built m anually/no longer in game
points_df[~points_df['match']]

In [ ]:
points_df = points_df[points_df['match']].copy()
points_df.drop('match', axis=1, inplace=True)

In [ ]:
points_df

In [ ]:
recipe_df

In [ ]:
# ingredients = recipe_df['ingredients'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().drop_duplicates()

In [ ]:
# misses = ingredients[~ingredients.isin(all_recipe_key_names)]

In [ ]:
# assume infinite power slugs, but wanna keep power shards --> remove every recipe that needs a power slug
# uranium/plut waste expected since only made as byproducts
# misses

In [ ]:
recipe_df.shape

In [ ]:
# def has_power_slug(ingr_list):
#     if not len(ingr_list):
#         return False
#     for i in ingr_list:
#         if 'power-slug' in i[0] in i[0]:
#             print('YO')
#             return True
#     return False

# recipe_df = recipe_df[~(recipe_df['ingredients'].apply(has_power_slug))]

In [ ]:
recipe_df.shape

In [ ]:
# products = recipe_df['products'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().drop_duplicates()

In [ ]:
# products[~products.isin(all_recipe_key_names)]

In [ ]:
recipe_df[recipe_df['key_name'].apply(lambda x: 'miner' in x)]

In [ ]:
recipe_df = recipe_df[~recipe_df['key_name'].isin(['automated-miner'])]

In [ ]:
recipe_df.shape

In [ ]:
recipe_df.reset_index(drop=True, inplace=True)

In [ ]:
recipe_df

In [ ]:
recipes_by_item = recipe_df.explode('products')
recipes_by_item['products'] = recipes_by_item['products'].apply(lambda x: x[0])
# recipes_by_item = recipes_by_item.groupby('products')['key_name'].apply(list).to_dict()
# for k, v in recipes_by_item.items():
#     print(k, v)
#     break

In [ ]:
points_df

In [ ]:
all_items = recipe_df['products'].explode().dropna().apply(lambda x: x[0]).drop_duplicates().reset_index(drop=True)

In [ ]:
all_items

In [ ]:
points_df['key_name'].isin(all_items).mean()

In [ ]:
# with pd.option_context('display.max_colwidth', 999):
#     display(recipe_df[recipe_df['key_name'].apply(lambda x: 'turbofuel' in x)])

In [ ]:
with pd.option_context('display.max_colwidth', 999):
    display(recipe_df[recipe_df['key_name'].apply(lambda x: 'alt-distilled-silica' in x)])

In [ ]:
# with pd.option_context('display.max_rows', 999):
#     display(points_df)

In [ ]:
points_df.tail(20)

In [ ]:
res

In [ ]:
recipe_df[recipe_df['key_name'].apply(lambda x: 'iron' in x)]

In [ ]:
sys.exit()

In [ ]:
for k, v in raw_recipe_info_dict.items():
    print(k, v)
    break

In [ ]:
recipes_by_item['iron-ingot']

In [ ]:
base_key_items

In [ ]:
recipe_dict = {k : [{'ingredients' : [], 'outputs' : k, 'power_rate' : 0}] for k in base_key_items}

In [ ]:
for k, v in recipe_dict.items():
    print(k, v)
    break

In [ ]:
recipes_by_item['iron-ingot']

In [ ]:
recipe_by_item_df = pd.DataFrame.from_dict({k : {'recipes' : v} for k, v in recipes_by_item.items()}, orient='index')
recipe_by_item_df = recipe_by_item_df.explode('recipes').reset_index()
recipe_by_item_df.columns = ['item', 'recipe_name']

In [ ]:
recipe_by_item_df

In [ ]:
recipe_df

In [ ]:
base_recipes

In [ ]:
recipe_df_full

In [ ]:
tmp = recipe_df_full['ingredients'].explode().dropna()#.apply(lambda x: x[0] if not pd.isnull(x) else None)
ingr = tmp[tmp.apply(len) > 0].apply(lambda x: x[0]).drop_duplicates()

In [ ]:
found = set(recipe_df_full['key_name'].tolist())

In [ ]:
recipe_df_full[recipe_df_full['key_name'].apply(lambda x: 'coal' in x)]

In [ ]:
ingr[~ingr.isin(found)]

In [ ]:
seen_bases = set()
def add_to_recipe_dict(key):
    # base items are in dict b/c they're base, but they have alternate recipes, so do this nonsense check
    if k in base_key_items_set and not k in seen_bases or not k in recipe_dict:
        # dont' always have reipce for things --> it should be a base --> return
        if not key in recipes_by_item:
            print(f'No recipes found for {key}; this ok for: SAM, [others?]')
            if key in base_key_items_set:
                seen_bases.add(k)
            return
        # what recipes make this item?
        this_item_recipes = recipes_by_item[key]
        # for each of them
        for recipe_name in this_item_recipes:
            # figure out what they are exactly
            recipe_deets = raw_recipe_info_dict[recipe_name]
            # figure out ways to make their children
            for ingredient in recipe_deets['ingredients']:
                # with the same nonsene child check as above, need to look at the child if it's base and we haven't checked it, or we haven'yt checked it generally
                if not ingredient in recipe_dict or (ingredient in recipe_dict and ingredient in base_key_items_set and not ingredient in seen_bases):
                    add_to_recipe_dict(ingredient)
                    
            
        
        if k in base_key_items_set:
            seen_bases.add(k)
        
        

In [ ]:
recipes_by_item['iron-ore']

In [ ]:
# add_to_recipe_dict('iron-ore')

In [ ]:
# recipe_dict['iron-ore']

In [ ]:
raw_recipe_info_dict['iron-ore-limestone']

In [ ]:
recipe_dict['reanimated-sam']

In [ ]:
recipes_by_item['reanimated-sam']

In [ ]:
raw_recipe_info_dict['reanimated-sam']

In [ ]:
recipe_dict['sam']

In [ ]:
recipe_dict['limestone']

In [ ]:
recipes_by_item['limestone']

In [ ]:
recipe_dict['']

In [ ]:
recipes_by_item['']

In [ ]:
points_df

In [ ]:
# eg = recipes_df[recipes_df['name'].apply(lambda x: 'iron rod' in x.lower())]

In [ ]:
# eg['produce'].iloc[0]

In [ ]:
# eg['ingredients'].iloc[0]

In [ ]:
from Levenshtein import distance as levenshtein_dist
from collections import defaultdict
lev_dist = lambda x,y: levenshtein_dist(x, y) / max(len(x), len(y))

In [ ]:
recipe_dict_raw = recipes_df.set_index('name')[['ingredients', 'produce']].to_dict()

In [ ]:
all_ios = set(recipes_df.explode('produce')['produce'].dropna().tolist()) | set(recipes_df.explode('ingredients')['ingredients'].dropna().tolist())

In [ ]:
with open('dumb_name_to_good.json', 'w') as f:
    json.dump({k : None for k in all_ios}, f, indent=4)

In [ ]:
recipes_df

In [ ]:
with pd.option_context('display.max_colwidth', 999):
    display(recipes_df[recipes_df['ingredients'].apply(lambda x: any(['TemporalProcessor' in y for y in x]))])

In [ ]:
with pd.option_context('display.max_colwidth', 999):
    display(recipes_df[recipes_df['produce'].apply(lambda x: any(['JumpingStilts' in y for y in x]))])

In [ ]:
points_df[points_df['item'].apply(lambda x: 'stilt' in x.lower())]

In [ ]:
points_df.shape

In [ ]:
for k, v in output_to_recipes.items():
    print(k, v)
    break

In [ ]:
def get_matching_items(key):
    return {k : v for k, v in output_to_recipes.items() if key.replace(' ', '').lower() in k.lower()}

In [ ]:
points_df['matched_recipes'] = points_df['item'].apply(get_matching_items)

In [ ]:
points_df['n_matched'] = points_df['matched_recipes'].apply(len)

In [ ]:
points_df.sort_values('n_matched')

In [ ]:
with pd.option_context('display.max_rows', 999):
    display(points_df[points_df['n_matched'] == 0])

In [ ]:
with pd.option_context('display.max_rows', 999, 'display.max_colwidth', 999):
    display(points_df[points_df['n_matched'] > 0].sort_values('n_matched'))

In [ ]:
points_df['item'].apply(get_matching_items)

In [ ]:

from scipy.spatial.distance import squareform, pdist

In [ ]:
dist_mat = squareform(pdist())

In [ ]:
points_info

In [ ]:
recipes_df['ingredients'].explode().tolist()

In [ ]:
schematics_df = pd.DataFrame.from_dict(raw_data['schematicsData'], orient='index')

In [ ]:
schematics_df

In [ ]:
is_alternate = recipes_df['className'].apply(lambda x: (x.split('/')[4] if x.count('/') > 4 else None) == 'AlternateRecipes')

In [ ]:
recipes_df[is_alternate]['name']

In [ ]:

recipes_df['className'].apply().value_counts()

In [ ]:
pd.DataFrame.from_dict(raw_data['schematicsData'], orient='index')['recipes'].shape

In [ ]:
building_df

In [ ]:
item_df

In [ ]:
recipes_df

In [ ]:
pd.DataFrame.from_dict(raw_data['buildingsCategories'], orient='index')

In [ ]:
greg

In [ ]:
df = pd.DataFrame.from_dict(load_json('working.json'), orient='index')

In [ ]:
keys = df[0].apply(lambda x: x.keys()).explode().dropna().unique()
for k in keys:
    df[k] = df[0].apply(lambda x: x[k])

In [ ]:
keys

In [ ]:
df

In [ ]:
df['className'].value_counts()

In [ ]:
points_df